In [1]:
# ~~~ Imports ~~~
import subprocess
import os
import hydra
from flows.utils.general_helpers import read_yaml_file
from flows.backends.api_info import ApiInfo
from flows.utils.general_helpers import find_replace_in_dict
from flows.flow_launchers import FlowLauncher
from flows.backends.api_info import ApiInfo
from flows import flow_verse
from flows import logging

logging.set_verbosity_info()

/Users/nicolasbaldwin/opt/miniconda3/envs/flows/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Defining Functions Which Reproduce The Steps Of The Quick Start Tutorial (no modifications required)

In [2]:
# ~~~ Convenience function that install the dependencies for the Flows in this notebook ~~~
# Normally, you would install the dependencies in your terminal, as necessary. We do it here for you convenience

DEPENDENCIES_DICT = {
    "ChatFlowModule": "aiflows/ChatFlowModule",
    "ReAct": "aiflows/ControllerExecutorFlowModule",
    "ChatInteractiveFlowModule": "aiflows/ChatInteractiveFlowModule",
    "ChatWithDemonstrationsFlowModule": "aiflows/ChatWithDemonstrationsFlowModule",
    "AutoGPTFlowModule": "aiflows/AutoGPTFlowModule",
    "VisionFlowModule": "aiflows/VisionFlowModule",
}

def download_all_flow_dependencies(flow):
    """Download all dependencies for a flow from FlowVerse and pip requirements. 
    
    Note that this is a convenience function for this tutorial and is not meant to work doesn't work for all Flows on FlowVerse.
    
    :param flow: The name of the Flow to download dependencies for
    :type flow: str
    """
    # ~~~ Load Flow dependecies from FlowVerse ~~~
    dependencies = [
        {"url": DEPENDENCIES_DICT[flow], "revision": "main"},
    ]
    flow_verse.sync_dependencies(dependencies)
    
    # path to pip requirements file
    path_to_requirements = os.path.join("flow_modules" ,DEPENDENCIES_DICT[flow], "pip_requirements.txt")

    # download dependencies
    subprocess.check_call(["pip", "install", "-r", path_to_requirements], stdout=subprocess.DEVNULL)

# ~~~ Load Flow dependecies from FlowVerse and pip requirments ~~~
for key in DEPENDENCIES_DICT.keys():
    download_all_flow_dependencies(key)

[2023-12-07 14:18:36,842][flows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/examples/quick_start/flow_modules...
[2023-12-07 14:18:36,847][flows.flow_verse.loading:563][INFO] - aiflows/ChatFlowModule:main will be fetched from remote


Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 44992.53it/s]


[2023-12-07 14:18:37,931][flows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing


[2023-12-07 14:18:51,708][flows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/examples/quick_start/flow_modules...
[2023-12-07 14:18:51,715][flows.flow_verse.loading:563][INFO] - aiflows/ControllerExecutorFlowModule:main will be fetched from remote


Fetching 14 files: 100%|██████████| 14/14 [00:00<00:00, 7434.83it/s]


[2023-12-07 14:18:52,952][flows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing


[2023-12-07 14:19:01,793][flows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/examples/quick_start/flow_modules...
[2023-12-07 14:19:01,797][flows.flow_verse.loading:563][INFO] - aiflows/ChatInteractiveFlowModule:main will be fetched from remote


...

[2023-12-07 14:19:10,665][flows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing


[2023-12-07 14:19:17,335][flows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/examples/quick_start/flow_modules...
[2023-12-07 14:19:17,340][flows.flow_verse.loading:563][INFO] - aiflows/AutoGPTFlowModule:main will be fetched from remote


Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 43893.88it/s]


[2023-12-07 14:19:18,098][flows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing


[2023-12-07 14:19:27,110][flows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/examples/quick_start/flow_modules...
[2023-12-07 14:19:27,114][flows.flow_verse.loading:563][INFO] - aiflows/VisionFlowModule:main will be fetched from remote


Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 62497.91it/s]


[2023-12-07 14:19:27,847][flows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing




In [3]:
def run_flow(flow, data, api_information):
    """Loads the Flow config, instantiates the Flow and runs inference.
    
    :param flow: The name of the flow to run
    :type flow: str
    :param data: The input data to the flow
    :type data: dict
    :param api_information: The api information for the flow
    :type api_information: ApiInfo
    """
    
    flow_url = DEPENDENCIES_DICT[flow]

    # ~~~ Load Flow config ~~~
    path_to_config = os.path.join( "flow_modules" ,flow_url, "demo.yaml")
    cfg = read_yaml_file(path_to_config)

    # set the API information
    cfg = find_replace_in_dict(cfg, key_to_find="api_infos", new_value= api_information , current_path="")
  
    # ~~~ Instantiate the Flow ~~~
    flow_with_interfaces = {
        "flow": hydra.utils.instantiate(cfg["flow"], _recursive_=False, _convert_="partial"),
        "input_interface": None,
        "output_interface": None
    }
    
    # ~~~ Run inference ~~~
    path_to_output_file = None
    # path_to_output_file = "output.jsonl"  # Uncomment this line to save the output to disk

    _, outputs = FlowLauncher.launch(
        flow_with_interfaces=flow_with_interfaces, data=data, path_to_output_file=path_to_output_file
    )

    # ~~~ Print the output ~~~
    flow_output_data = outputs[0]
    print(flow_output_data)
    

### Setting you API Information

Set your API information here (copy-paste it)

In [4]:

#OpenAI backend
api_key = "" # copy paste your api key here
api_information = [ApiInfo(backend_used="openai", api_key=api_key)]

# Azure backend
# api_key = "" # copy paste your api key here
# api_base = "" # copy paste your api base here
# api_version = "" #copypase your api base here
# api_information = ApiInfo(backend_used = "azure",
#                           api_base =api_base,
#                           api_key = api_version,
#                           api_version =  api_version )

### Pick a Flow, some data and run it!

Set the `Flow` and `data` values, then run the cell! Examples for each flow from the FlowVerse are provided in the cell bellow. Uncomment the flow and data you wish to run. Here is the list of flows available for you to run:

* [ChatFlowModule](https://huggingface.co/aiflows/ChatFlowModule)

* [ReAct](https://huggingface.co/aiflows/ControllerExecutorFlowModule)

* [AutoGPTFlowModule](https://huggingface.co/aiflows/AutoGPTFlowModule)

* [ChatInteractiveFlowModule](https://huggingface.co/aiflows/ChatInteractiveFlowModule)

* [ChatWithDemonstrationsFlowModule](https://huggingface.co/aiflows/ChatWithDemonstrationsFlowModule)

* [VisionFlowModule](https://huggingface.co/aiflows/VisionFlowModule)


In [5]:
# Example for a simple chatbot
# [ChatFlowModule](https://huggingface.co/aiflows/ChatFlowModule)

flow = "ChatFlowModule"  
data = {"id": 0, "question": "Who was the NBA champion in 2023?"} 
run_flow(flow, data, api_information)

[2023-12-07 14:19:43,136][flows.flow_launchers.flow_API_launcher:194][INFO] - Running inference for ID (sample 0): 0
[{'api_output': "I'm sorry, but as an AI language model, I don't have access to real-time information or the ability to predict future events. As of now, I cannot provide you with the answer to who the NBA champion was in 2023. I recommend checking reliable sports news sources or conducting an internet search for the most up-to-date information."}]


In [6]:
# Example for ReAct (the simplest instance of a ControllerExecutorFlowModule)
flow = "ReAct"
data = {"id": 0, "goal": "Who was the NBA champion in 2023?"}
run_flow(flow, data, api_information)

[2023-12-07 14:19:46,287][flows.flow_verse.loading:775][INFO] - [aiflows.ControllerExecutorFlowModule] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/examples/quick_start/flow_modules...


[2023-12-07 14:19:46,496][flows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:main already synced, skip
[2023-12-07 14:19:46,500][flows.flow_verse.loading:825][INFO] - [aiflows.ControllerExecutorFlowModule] finished syncing


[2023-12-07 14:19:46,519][flows.base_flows.circular:94][DEBUG] - output_msg_payload_processor [ControllerExecutorFlow.detect_finish_or_continue] registered
[2023-12-07 14:19:46,585][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/CompositeFlow.yaml.
[2023-12-07 14:19:46,758][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 14:19:46,957][flows.base_flows.abstract:70][DEBUG] - Flow ControllerFlow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── True                                                                                                       
├── name
│   └── ControllerFlow                                                                                             
├── description
│   └── Proposes the next action to take towards achieving the goal, and prepares the input for the executor.      
├── n_api_retries
│   └── 6                                                                                                          
├── wait_time_between_retries
│   └── 20                                                                                                         
├── system_name
│   └── system                                                                                                     
├── user_name
│   └── user                                                                                                       
├── assistant_name
│   └── assistant                                                                                                  
├── backend
│   └── {'_target_': 'flows.backends.llm_lite.LiteLLMBackend', 'api_infos': [{'backend_used': 'openai', 'api_versio
├── system_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'You are a smart AI assistant. \n\nYour decis
├── init_human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'Here is the goal you need to achieve:\n{{goa
├── human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'Here is the response to your last action:\n{
├── input_interface_initialized
│   └── ['observation']                                                                                            
├── query_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt'}                                                          
├── previous_messages
│   └── {'first_k': None, 'last_k': None}                                                                          
├── output_interface
│   └── ['thought', 'reasoning', 'plan', 'criticism', 'speak', 'command', 'command_args']                          
├── input_interface_non_initialized
│   └── ['goal']                                                                                                   
├── finish
│   └── {'description': 'Signal that the objective has been satisfied, and returns the answer to the user.', 'input
└── commands
    └── {'wiki_search': {'description[2;38;2

[2023-12-07 14:19:47,129][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/CompositeFlow.yaml.
[2023-12-07 14:19:47,139][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 14:19:47,154][flows.base_flows.abstract:70][DEBUG] - Flow WikiSearchAtomicFlow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── name
│   └── WikiSearchAtomicFlow                                                                                       
├── description
│   └── A Flow that queries the wikipedia API for a page content.                                                  
├── keep_raw_response
│   └── False                                                                                                      
├── lang
│   └── en                                                                                                         
├── top_k_results
│   └── 5                                                                                                          
├── doc_content_chars_max
│   └── 3000                                                                                                       
├── input_interface
│   └── ['search_term']                                                                                            
└── output_interface
    └── ['wiki_content']                                                                                           

[2023-12-07 14:19:47,193][flows.base_flows.abstract:70][DEBUG] - Flow unknown_name instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── input_interface
│   └── ['branch', 'branch_input_data']                                                                            
├── output_interface
│   └── ['branch_output_data']                                                                                     
└── subflows_config
    └── {'wiki_search': {'_target_': 'aiflows.ControllerExecutorFlowModule.WikiSearchAtomicFlow.instantiate_from_de

[2023-12-07 14:19:47,228][flows.base_flows.abstract:70][DEBUG] - Flow CtrlEx instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── max_rounds
│   └── 30                                                                                                         
├── early_exit_key
│   └── EARLY_EXIT                                                                                                 
├── topology
│   └── [{'goal': 'Select the next action and prepare the input for the executor.', 'input_interface': {'_target_':
├── name
│   └── CtrlEx                                                                                                     
├── description
│   └── ControllerExecutor (i.e., MRKL, ReAct) interaction implementation with Flows that approaches the problem so
├── input_interface
│   └── ['goal']                                                                                                   
├── output_interface
│   └── ['answer', 'status']                                                                                       
└── subflows_config
    └── {'Controller': {'_target_': 'aiflows.ControllerExecutorFlowModule.ControllerAtomicFlow.instantiate_from_def

[2023-12-07 14:19:47,378][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `Controller` ~~~
{
    "message_id": "dc809b3d-75e9-4624-b0e7-afa576ca5a9c",
    "created_at": "2023-12-07 13:19:47.378554000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "Controller"
}
[2023-12-07 14:19:47,384][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `wiki_search` ~~~
{
    "message_id": "bc1bdd0b-a484-4fb1-b4ff-f1f7279946d9",
    "created_at": "2023-12-07 13:19:47.384401000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "wiki_search"
}
[2023-12-07 14:19:47,386][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `Executor` ~~~
{
    "message_id": "bdc7bf2d

In [9]:
# Example for AutoGPTFlowModule
flow = "AutoGPTFlowModule"
data = {"id": 0, "goal": "Who was the NBA champion in 2023?"}
run_flow(flow, data, api_information)

[2023-12-07 14:24:53,485][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/CompositeFlow.yaml.


[2023-12-07 14:24:53,618][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 14:24:53,833][flows.base_flows.abstract:70][DEBUG] - Flow ControllerFlow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── True                                                                                                       
├── name
│   └── ControllerFlow                                                                                             
├── description
│   └── Proposes the next action to take towards achieving the goal, and prepares the input for the executor.      
├── n_api_retries
│   └── 6                                                                                                          
├── wait_time_between_retries
│   └── 20                                                                                                         
├── system_name
│   └── system                                                                                                     
├── user_name
│   └── user                                                                                                       
├── assistant_name
│   └── assistant                                                                                                  
├── backend
│   └── {'_target_': 'flows.backends.llm_lite.LiteLLMBackend', 'api_infos': [{'backend_used': 'openai', 'api_versio
├── system_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'You are a smart AI assistant. \n\nYour decis
├── init_human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'Here is the goal you need to achieve:\n{{goa
├── human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'Here is the response to your last action:\n{
├── input_interface_initialized
│   └── ['observation', 'human_feedback']                                                                          
├── query_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt'}                                                          
├── previous_messages
│   └── {'first_k': None, 'last_k': None}                                                                          
├── output_interface
│   └── ['thought', 'reasoning', 'plan', 'criticism', 'speak', 'command', 'command_args']                          
├── input_interface_non_initialized
│   └── ['goal']                                                                                                   
├── finish
│   └── {'description': 'Signal that the objective has been satisfied, and returns the answer to the user.', 'input
└── commands
    └── {'wiki_search'[2;48

[2023-12-07 14:24:53,922][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/CompositeFlow.yaml.
[2023-12-07 14:24:53,929][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 14:24:53,942][flows.base_flows.abstract:70][DEBUG] - Flow WikiSearchAtomicFlow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── name
│   └── WikiSearchAtomicFlow                                                                                       
├── description
│   └── A Flow that queries the wikipedia API for a page content.                                                  
├── keep_raw_response
│   └── False                                                                                                      
├── lang
│   └── en                                                                                                         
├── top_k_results
│   └── 5                                                                                                          
├── doc_content_chars_max
│   └── 3000                                                                                                       
├── input_interface
│   └── ['search_term']                                                                                            
└── output_interface
    └── ['wiki_content']                                                                                           

...

[2023-12-07 14:24:54,455][flows.base_flows.abstract:70][DEBUG] - Flow AutoGPTFlow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── max_rounds
│   └── 30                                                                                                         
├── early_exit_key
│   └── EARLY_EXIT                                                                                                 
├── topology
│   └── [{'goal': 'Retrieve relevant information from memory.', 'input_interface': {'_target_': 'AutoGPTFlow.prepar
├── name
│   └── AutoGPTFlow                                                                                                
├── description
│   └── An example implementation of AutoGPT with Flows.                                                           
├── input_interface
│   └── ['goal']                                                                                                   
├── output_interface
│   └── ['answer', 'status']                                                                                       
└── subflows_config
    └── {'Controller': {'_target_': 'aiflows.ControllerExecutorFlowModule.ControllerAtomicFlow.instantiate_from_def

[2023-12-07 14:24:54,550][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `Controller` ~~~
{
    "message_id": "ae808372-3907-449a-87cf-d74168fa7ac3",
    "created_at": "2023-12-07 13:24:54.550417000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "Controller"
}
[2023-12-07 14:24:54,554][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `wiki_search` ~~~
{
    "message_id": "86c3bb53-5e67-49fa-8d9e-3b7dbf364132",
    "created_at": "2023-12-07 13:24:54.554362000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "wiki_search"
}
[2023-12-07 14:24:54,556][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `Executor` ~~~
{
    "message_id": "48f86857

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


[2023-12-07 14:25:36,839][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageNamespaceOnly (UpdateMessage_NamespaceReset): `Memory` ~~~
{
    "message_id": "6af928fc-fbf8-4c86-b032-addceeea81fb",
    "created_at": "2023-12-07 13:25:36.839684000",
    "message_type": "UpdateMessage_NamespaceReset",
    "created_by": "Memory",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "Memory"
}
[2023-12-07 14:25:36,841][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ OutputMessage: `Memory` --> `AutoGPTFlow` ~~~
{
    "message_id": "dcb0f758-558f-4d32-9947-40a2f49b254a",
    "created_at": "2023-12-07 13:25:36.839462000",
    "message_type": "OutputMessage",
    "created_by": "Memory",
    "data": {
        "output_data": {
            "retrieved": [
                [
                    "\n            == Goal == \n            Who was the NBA champion in 2023?\n            \n            == Command ==\n            wiki_search\n            == Args\n            

In [10]:
# Example for ChatInteractiveFlowModule
flow = "ChatInteractiveFlowModule"
data = {"id": 0} # you will be able to ask questions once it is running
run_flow(flow, data, api_information)

[2023-12-07 14:26:20,333][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/CompositeFlow.yaml.
[2023-12-07 14:26:20,435][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 14:26:20,533][flows.base_flows.abstract:70][DEBUG] - Flow ChatAtomicFlow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── True                                                                                                       
├── name
│   └── ChatAtomicFlow                                                                                             
├── description
│   └── Flow which uses as tool an LLM though an API                                                               
├── n_api_retries
│   └── 6                                                                                                          
├── wait_time_between_retries
│   └── 20                                                                                                         
├── system_name
│   └── system                                                                                                     
├── user_name
│   └── user                                                                                                       
├── assistant_name
│   └── assistant                                                                                                  
├── backend
│   └── {'_target_': 'flows.backends.llm_lite.LiteLLMBackend', 'api_infos': [{'backend_used': 'openai', 'api_versio
├── system_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt'}                                                          
├── init_human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt'}                                                          
├── human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': '{{query}}', 'input_variables': ['query']}   
├── input_interface_initialized
│   └── ['query']                                                                                                  
├── query_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt'}                                                          
├── previous_messages
│   └── {'first_k': None, 'last_k': None}                                                                          
├── output_interface
│   └── ['api_output']                                                                                             
└── input_interface_non_initialized
    └── []                                                                                                         

[2023-12-07 14:26:20,774][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 14:26:20,794][flows.base_flows.abstract:70][DEBUG] - Flow unknown_name instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── end_of_input_string
│   └── EOI                                                                                                        
├── description
│   └── Reads input from the user's standard input.                                                                
├── query_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': '{{api_output}}\n\nTo end an Interaction, typ
├── request_multi_line_input_flag
│   └── False                                                                                                      
├── input_interface
│   └── ['api_output']                                                                                             
└── output_interface
    └── ['human_input']                                                                                            

[2023-12-07 14:26:20,974][flows.base_flows.abstract:70][DEBUG] - Flow ChatInteractiveFlow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── max_rounds
│   └── 5                                                                                                          
├── early_exit_key
│   └── end_of_interaction                                                                                         
├── topology
│   └── [{'goal': 'Query the assistant', 'input_interface': {'_target_': 'flows.interfaces.KeyInterface', 'addition
├── name
│   └── ChatInteractiveFlow                                                                                        
├── description
│   └── Flow that enables chatting between a ChatAtomicFlow and a user providing the input.                        
├── input_interface
│   └── {'_target_': 'flows.interfaces.KeyInterface'}                                                              
├── output_interface
│   └── ['end_of_interaction', 'answer']                                                                           
└── subflows_config
    └── {'Assistant': {'_target_': 'aiflows.ChatFlowModule.ChatAtomicFlow.instantiate_from_default_config', 'backen

[2023-12-07 14:26:21,406][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `Assistant` ~~~
{
    "message_id": "1125d876-5d95-46ec-8f02-7775f6e07a9d",
    "created_at": "2023-12-07 13:26:21.406129000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "Assistant"
}
[2023-12-07 14:26:21,410][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `User` ~~~
{
    "message_id": "712f9be4-de62-4c3b-8052-7f6a190889a8",
    "created_at": "2023-12-07 13:26:21.410767000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "User"
}
[2023-12-07 14:26:21,413][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `ChatInteractiveFlow` ~~~
{
    "message_id": "ea6386bc-5cb3

In [11]:
# Example for ChatWithDemonstrationsFlowModule
flow = "ChatWithDemonstrationsFlowModule"
data = {"id": 0, "question": "What's the capital of France?"}  # This can be a list of samples
run_flow(flow, data, api_information)

[2023-12-07 14:26:40,925][flows.flow_verse.loading:775][INFO] - [aiflows.ChatWithDemonstrationsFlowModule] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/examples/quick_start/flow_modules...


[2023-12-07 14:26:41,083][flows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:main already synced, skip
[2023-12-07 14:26:41,210][flows.flow_verse.loading:825][INFO] - [aiflows.ChatWithDemonstrationsFlowModule] finished syncing


[2023-12-07 14:26:41,278][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/CompositeFlow.yaml.
[2023-12-07 14:26:41,485][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 14:26:41,660][flows.base_flows.abstract:70][DEBUG] - Flow DemonstrationAtomicFlow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── name
│   └── DemonstrationAtomicFlow                                                                                    
├── description
│   └── A flow which returns Demonstrations                                                                        
├── data
│   └── [{'query_data': {'query': 'What is the capital of Turkey?'}, 'response_data': {'response': 'Istambul, my si
├── params
│   └── {'data_dir': None, 'demonstrations_id': 'my_sir_demo', 'ids_to_keep': None, 'demonstrations_k': None}      
├── query_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'Answer the following question: {{query}}', '
└── response_prompt_template
    └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': '{{response}}', 'input_variables': ['response

[2023-12-07 14:26:41,845][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 14:26:42,128][flows.base_flows.abstract:70][DEBUG] - Flow SimpleQA_Flow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── True                                                                                                       
├── name
│   └── SimpleQA_Flow                                                                                              
├── description
│   └── Flow which uses as tool an LLM though an API                                                               
├── n_api_retries
│   └── 6                                                                                                          
├── wait_time_between_retries
│   └── 20                                                                                                         
├── system_name
│   └── system                                                                                                     
├── user_name
│   └── user                                                                                                       
├── assistant_name
│   └── assistant                                                                                                  
├── backend
│   └── {'_target_': 'flows.backends.llm_lite.LiteLLMBackend', 'api_infos': [{'backend_used': 'openai', 'api_versio
├── system_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'You are a helpful chatbot that truthfully an
├── init_human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'Answer the following question: {{question}}'
├── human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': '{{query}}', 'input_variables': ['query']}   
├── input_interface_initialized
│   └── ['query']                                                                                                  
├── query_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt'}                                                          
├── previous_messages
│   └── {'first_k': None, 'last_k': None}                                                                          
├── output_interface
│   └── ['api_output']                                                                                             
└── input_interface_non_initialized
    └── ['question']                                                                                               

[2023-12-07 14:26:42,278][flows.base_flows.abstract:70][DEBUG] - Flow SimpleQA_Flow_with_Demonstrations instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── max_rounds
│   └── 1                                                                                                          
├── early_exit_key
│   └── EARLY_EXIT                                                                                                 
├── topology
│   └── [{'goal': 'Get Demonstrations', 'input_interface': {'_target_': 'flows.interfaces.KeyInterface'}, 'flow': '
├── name
│   └── SimpleQA_Flow_with_Demonstrations                                                                          
├── description
│   └── A sequential flow that answers questions with demonstrations                                               
├── subflows_config
│   └── {'demonstration_flow': {'_target_': 'aiflows.ChatWithDemonstrationsFlowModule.DemonstrationsAtomicFlow.inst
├── input_interface_initialized
│   └── ['query']                                                                                                  
├── output_interface
│   └── ['answer']                                                                                                 
└── input_interface
    └── ['questions']                                                                                              

[2023-12-07 14:26:42,430][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `demonstration_flow` ~~~
{
    "message_id": "f0541220-a6db-41d9-b517-a315434f1392",
    "created_at": "2023-12-07 13:26:42.430576000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "demonstration_flow"
}
[2023-12-07 14:26:42,436][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `chat_flow` ~~~
{
    "message_id": "b65bc5a3-d196-4b0c-aa1d-a52846b24682",
    "created_at": "2023-12-07 13:26:42.435991000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "chat_flow"
}
[2023-12-07 14:26:42,442][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `SimpleQA_Flow_with_Demonstratio

In [12]:
# Example for VisionFlowModule
flow = "VisionFlowModule"
url_image = {"type": "url",
            "image": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"}
data = {"id": 0, "query": "What’s in this image?", "data": {"images": [url_image]}}  # This can be a list of samples
run_flow(flow, data, api_information)

[2023-12-07 14:27:01,439][flows.flow_verse.loading:775][INFO] - [aiflows.VisionFlowModule] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/examples/quick_start/flow_modules...
[2023-12-07 14:27:01,566][flows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:main already synced, skip
[2023-12-07 14:27:01,570][flows.flow_verse.loading:825][INFO] - [aiflows.VisionFlowModule] finished syncing


[2023-12-07 14:27:03,562][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 14:27:03,696][flows.base_flows.abstract:70][DEBUG] - Flow Demo Vision Flow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── True                                                                                                       
├── name
│   └── Demo Vision Flow                                                                                           
├── description
│   └── A flow that, given a textual input, and a set of images and/or videos, generates a textual output.         
├── n_api_retries
│   └── 6                                                                                                          
├── wait_time_between_retries
│   └── 20                                                                                                         
├── system_name
│   └── system                                                                                                     
├── user_name
│   └── user                                                                                                       
├── assistant_name
│   └── assistant                                                                                                  
├── backend
│   └── {'_target_': 'flows.backends.llm_lite.LiteLLMBackend', 'api_infos': [{'backend_used': 'openai', 'api_versio
├── system_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'You are a helpful chatbot that truthfully an
├── init_human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': '{{query}}', 'input_variables': ['query'], 'p
├── human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': '{{query}}', 'input_variables': ['query']}   
├── input_interface_initialized
│   └── ['query', 'data']                                                                                          
├── query_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt'}                                                          
├── previous_messages
│   └── {'first_k': None, 'last_k': None}                                                                          
├── output_interface
│   └── ['api_output']                                                                                             
├── input_interface
│   └── ['query', 'data']                                                                                          
└── input_interface_non_initialized
    └── ['question', 'data']                                                                                       

[2023-12-07 14:27:03,795][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `Demo Vision Flow` ~~~
{
    "message_id": "b5f3a1fd-9a2d-49e3-804b-2933d77db1ad",
    "created_at": "2023-12-07 13:27:03.795155000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "Demo Vision Flow"
}
[2023-12-07 14:27:03,802][flows.flow_launchers.flow_API_launcher:194][INFO] - Running inference for ID (sample 0): 0
[2023-12-07 14:27:03,812][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ InputMessage: `Launcher` --> `Demo Vision Flow` ~~~
{
    "message_id": "057ab4df-a4a3-45be-b426-80d02ac4ffff",
    "created_at": "2023-12-07 13:27:03.811479000",
    "message_type": "InputMessage",
    "created_by": "Launcher",
    "data": {
        "id": 0,
        "query": "What\u2019s in this image?",
        "data": {
            "images": [
                {
             